In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [5]:
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}")

Using device: mps


Reading the dataset and resizing the images

In [7]:
data_path = './data/parking-training-data/'

In [21]:
transform = transforms.Compose([
    transforms.Resize((31, 69)),  # Resize images to 31 x 69
    transforms.ToTensor(),         # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize (adjust mean & std if grayscale)
])

In [22]:
dataset = datasets.ImageFolder(root=data_path, transform=transform)

In [23]:
print("Class to Index Mapping:", dataset.class_to_idx)  # {'empty': 0, 'non-empty': 1}

Class to Index Mapping: {'empty': 0, 'not_empty': 1}


In [24]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset)  - train_size

In [25]:
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [26]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [29]:
images, labels = next(iter(train_loader))
print(f"Batch of images shape: {images.shape}") # (batch_size, channels, height, width)

Batch of images shape: torch.Size([32, 3, 31, 69])


Model

In [72]:
class ParkingModel(nn.Module):
    def __init__(self):
        super(ParkingModel, self).__init__()
        self.cnn1 = nn.Conv2d(3,16,kernel_size=3,stride=1, padding = 1)
        self.maxpool1 = nn.MaxPool2d(3, stride=2)
        self.cnn2 = nn.Conv2d(16,64,kernel_size=2,stride=1, padding = 1)
        self.maxpool2 = nn.MaxPool2d(2, stride=2)
        self.linear = nn.Linear(64 * 8 * 17, 128)
        self.linear2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.maxpool1(F.relu(self.cnn1(x)))
        x = self.maxpool2(F.relu(self.cnn2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.linear(x))
        return self.linear2(x)

In [73]:
model = ParkingModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Training

In [74]:
epochs = 5

for epoch in range(epochs):
    model.train() 
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}")


Epoch [1/5], Loss: 0.0506
Epoch [2/5], Loss: 0.0032
Epoch [3/5], Loss: 0.0002
Epoch [4/5], Loss: 0.0001
Epoch [5/5], Loss: 0.0000


In [78]:
correct = 0
total = 0

model.eval()

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()  # Count correct predictions

accuracy = correct / total * 100
print(f'Validation Accuracy: {accuracy:.2f}%')

Validation Accuracy: 100.00%


In [79]:
torch.save(model.state_dict(), "parking_model_weights.pth")